In [2]:
try:
    print('Downloading needed packages...')
    !conda install -c conda-forge geocoder --yes
    !conda install -c conda-forge folium --yes
    print('Success: All lib downloaded')
except:
    print('Error')

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    orderedset-2.0             |           py35_0         685 KB  conda-forge
    ratelim-0.1.6              |           py35_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_0       conda-forge
    orderedset:

In [1]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

,Borough,Neighborhood,Latitude,Longitude


In [13]:
neighborhoods_data = newyork_data['features']

In [14]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [16]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [17]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [20]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [21]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [23]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [25]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [26]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [27]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [28]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [29]:
CLIENT_ID = 'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L' # your Foursquare ID
CLIENT_SECRET = '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L
CLIENT_SECRET:1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33


In [30]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [31]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [77]:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L', 
            '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33', 
            20190530, 
            40.8765, 
            -73.9106, 
            500, 
            100)

In [78]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf8bbf76a60714941992c8a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L', 
            '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33', 
            20190530, 
            40.8765, 
            -73.9106, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [81]:
print(manhattan_venues.shape)
manhattan_venues.head()

(1040, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [82]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,26,26,26,26,26,26
Carnegie Hill,26,26,26,26,26,26
Central Harlem,26,26,26,26,26,26
Chelsea,26,26,26,26,26,26
Chinatown,26,26,26,26,26,26
Civic Center,26,26,26,26,26,26
Clinton,26,26,26,26,26,26
East Harlem,26,26,26,26,26,26
East Village,26,26,26,26,26,26


In [83]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 23 uniques categories.


In [84]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supermarket,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
manhattan_onehot.shape

(1040, 24)

In [86]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Bakery,Bank,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supermarket,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,Battery Park City,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
1,Carnegie Hill,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
2,Central Harlem,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
3,Chelsea,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
4,Chinatown,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
5,Civic Center,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
6,Clinton,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
7,East Harlem,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
8,East Village,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
9,Financial District,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462


In [87]:
manhattan_grouped.shape

(40, 24)

In [88]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Central Harlem----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Chinatown----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Civic Cente

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
1,Carnegie Hill,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
2,Central Harlem,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
3,Chelsea,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
4,Chinatown,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store


In [91]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,0,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
3,Manhattan,Inwood,40.867684,-73.921210,0,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
1,Chinatown,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
2,Washington Heights,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
3,Inwood,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
4,Hamilton Heights,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
5,Manhattanville,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
6,Central Harlem,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
7,East Harlem,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
8,Upper East Side,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
9,Yorkville,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store


In [95]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [96]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [97]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [98]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
